## Task ba9d

In [1]:
def read_data(fname='rosalind_ba9d.txt'):
    with open(fname, 'r') as f:
        s = f.readline().strip()
    return s

In [2]:
def get_list_of_suffixes(s):
    # return suffix as well as its index and 2 placeholders for 2 rank variables
    return [s[i:] for i in range(len(s))]

In [47]:
class Node(object):
    def __init__(self, level=0, parent=None):
        self.value = {}
        self.parent = parent
        self.children = {}
        if parent is not None:
            self.level = self.parent.level+1
    def set_parent(self, parent):
        self.parent = parent
        self.level = self.parent.level+1
        
# compressed trie structure (means value can be multiple letters)
class Trie(object):
    def __init__(self):
        self.root = Node(parent=None)
        self.root.level = 0
        
    def insert(self, word):
        cur_node = self.root
        i = 0
        
        while i < len(word) and word[i] in cur_node.value:
            j = 0
            idx = word[i]
            val = cur_node.value[idx]

            while j < len(val) and i < len(word) and val[j] == word[i]:
                i += 1
                j += 1
                
            if j == len(val):
                cur_node = cur_node.children[idx]
                cur_node.parent = cur_node
            else:
                if i != len(word):
                    rest_val = val[j:]
                    rest_word = word[i:]
                    
                    new_node = Node(parent=cur_node)
                    ex_child = cur_node.children[idx]
                    cur_node.value[idx] = val[:j]
                    cur_node.children[idx] = new_node
                    new_node.children[rest_val[0]] = ex_child
                    new_node.children[rest_val[0]].set_parent(new_node)
                    new_node.value[rest_val[0]] = rest_val
                    new_node.value[rest_word[0]] = rest_word
                    new_node.children[rest_word[0]] = Node(parent=new_node)
                    new_node.children[rest_word[0]].set_parent(new_node)
#                     print(new_node.level, new_node.children[rest_val[0]].level)
#                     print(new_node.level, new_node.children[rest_word[0]].level)
#                     print(cur_node.level, cur_node.children[idx].level)

                else:
                    ex_child = cur_node.children[idx]
                    new_node = Node(parent=cur_node)
                    rest_val = val[j:]
                    cur_node.value[idx] = val[:j]
                    cur_node.children[idx] = new_node
                    cur_node.children[idx].set_parent(cur_node)

#                     ex_child.level = ex_child.level + 1
                    new_node.children[rest_val[0]] = ex_child
                    new_node.value[rest_val[0]] = rest_val
                    new_node.children[rest_val[0]].set_parent(new_node)
#                     print(new_node.level, cur_node.level)
#                     print(cur_node.level, cur_node.children[idx].level)

                return

        if i < len(word):            
            cur_node.value[word[i]] = word[i:]
            cur_node.children[word[i]] = Node(parent=cur_node)
#             print(cur_node.level, cur_node.children[word[i]].level)
#             if cur_node.children[word[i]].level == 3:
#                 print(cur_node.value, cur_node.children[word[i]].value)

    def reduce_level(self, node):
        node.level = node.level - 1
        if node.children != {}:
            for k, child in node.children.items():
                self.reduce_level(child)

    def _compress_tree(self, node, k):
        if node.children == {}:
            return
        if len(list(node.children.values())) > 1:
            for k, child in node.children.items():
                self._compress_tree(child, k)
        else:
            k2 = list(node.children.keys())[0]
            child = node.children[k2]
            node.parent.value[k] = node.parent.value[k] + node.value[k2]
            node.parent.children[k] = child
            self.reduce_level(child)
            node.parent.children[k].set_parent(node.parent)
            self._compress_tree(node, k2)
            
    def compress_tree(self):
        return self._compress_tree(self.root, None)

    def _print_tree(self, node):
        
        if node.children == {}:
            print(node.level, node.value)
        else:
            for child in node.children.values():
                print(child.level, child.value)
                self._print_tree(child)
                
    def print_tree(self):
        print(self.root.level, self.root.value)
        # fix some cases where level is incorrect:
        self._print_tree(self.root)
        
    def _validate(self, node):
        
        if node.children == {}:
            if node.level == node.parent.level:
                node.level = node.level + 1
        else:
            for child in node.children.values():
                self._validate(child)
                
    def validate(self):
        # fix some cases where level is incorrect:
        self._validate(self.root)
        
    def insert_list(self, lst):
        for word in lst:
            self.insert(word)
        self.validate()

    def find(self, word):
        """
        Returns if the word is in the trie.
        :type word: str
        :rtype: bool
        """
        i = 0
        cur_node = self.root

        while i < len(word) and word[i] in cur_node.value:
            j = 0
            idx = word[i]
            val = cur_node.value[idx]

            while j < len(val) and i < len(word):
                if word[i] != val[j]:
                    return False
                i += 1
                j += 1

            if j == len(val) and len(word) >= i:
                cur_node = cur_node.children[idx]
            else:
                return False

        return i == len(word) and cur_node.leaf
    
    def get_paths_recursive(self, node, prev_path):
        global paths
        if node.value == {}:
#             paths.append((prev_path, node.level))
            return 
        for k, next_node in node.children.items():
            paths.append((prev_path, next_node.level))
            if next_node.value != {}:
                self.get_paths_recursive(next_node, prev_path + node.value[k])
    
    def get_deepest_path(self):
        cur_node = self.root
        global paths
        paths = []
        self.get_paths_recursive(cur_node, '')
#         print(paths)
        return sorted(paths, key=lambda x: (len(x[0]), x[1]))[-1][0]
        

In [48]:
def main(fname='rosalind_ba9d.txt'):
    s = read_data(fname)
    suf_lst = get_list_of_suffixes(s+'$')
    trie = Trie()
    trie.insert_list(suf_lst)
#     trie.print_tree()
    trie.compress_tree()
#     trie.print_tree()
    print(trie.get_deepest_path())
    return trie

In [49]:
tr = main('sample_data/ba9d/sample.txt')

TATCGTT


In [50]:
tr = main('sample_data/ba9d/input_2.txt')

GAGAG


In [52]:
tr.print_tree()

0 {'G': 'G', 'A': 'AG', '$': '$'}
1 {'A': 'AG', '$': '$'}
2 {'A': 'AG', '$': '$'}
2 {'A': 'AG$', '$': '$'}
3 {}
3 {}
3 {}
3 {}
3 {}
3 {}
2 {}
2 {}
1 {'A': 'AG', '$': '$'}
2 {'A': 'AG$', '$': '$'}
3 {}
3 {}
3 {}
3 {}
2 {}
2 {}
1 {}
1 {}


In [54]:
for i in range(1, 7):
    print('-------------------------')
    print(open(f'sample_data/ba9d/output_{i}.txt').readline())
    main(f'sample_data/ba9d/input_{i}.txt')
    

-------------------------
TATCGTT
TATCGTT
-------------------------
GAGAG
GAGAG
-------------------------
AG or CT (you will not be penalized for having one over the other, but make sure you only output one).
CT
-------------------------
AA
AA
-------------------------
nan

-------------------------
TTTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAAG
TTTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAAG


In [56]:
# driver code
# main()